In [1]:
year = "2021"
filing_type = "10-K"

start = year+"-01-01"
end = year+"-12-31"

In [1]:
from data_extraction import get_data

full_data = get_data(
    "TSLA",
    "2021",
    filing_type="10-Q",
    include_amends=True,
    num_workers=8
)

Started for TSLA


In [16]:
# len(full_data)
# full_data

In [ ]:
"\xa0","\n","\t",

In [3]:
for i in full_data['TSLA']:
    print(i['year'])
    print(i['filing_type'])

202109
10-Q
202106
10-Q
202103
10-Q


In [22]:
documents = []
metadata = []

for tic_data in full_data['TSLA']:
    # print(tic_data)
    curr_year = tic_data["year"]
    ticker = tic_data["ticker"]
    filing_type = tic_data["filing_type"]

    for section, section_text in tic_data["all_texts"].items():
        documents.append(section_text)
        metadata.append(
            {
                "year": curr_year,
                "ticker": ticker,
                "section": section,
                "filing_type": filing_type,
            }
        )

In [1]:
from sec_filings import SECExtractor
from data_extraction import multiprocess_run
def get_data_(
    ticker: str,
    year: str,
    filing_type: str,
    include_amends: bool = True,
    num_workers: int = 8,
    quarters:str = "ALL"
):
    assert filing_type in [
        "10-K",
        "10-Q",
    ], "The supported document types are 10-K and 10-Q"

    assert quarters in ["ALL","Q3","Q2","Q1"], "Please provide a valid quarter name"

    if filing_type == "10-K":
        year = str(int(year) + 1)
        start = year + "-01-01"
        end = year + "-12-31"
        amount = 1
    elif filing_type == "10-Q":
        if quarters == "ALL":
            start = year + "-01-01"
            end = str(int(year)+1) + "-02-20"
            amount = 3
        elif quarters == "Q3":
            start = year + "-07-01"
            end = str(int(year)+1) + "-02-20"
            amount = 1
        elif quarters == "Q2":
            start = year+"-04-01"
            end = year + "-12-31"
            amount = 1
        elif quarters == "Q1":
            start = year+"-01-01"
            end = year+"-05-01"
            amount = 1

    se = SECExtractor(
        [ticker],
        amount=amount,
        filing_type=filing_type,
        include_amends=include_amends,
        start_date=start,
        end_date=end,
    )

    full_data = multiprocess_run(se, ticker, num_workers)

    return full_data


for quarter in ["ALL","Q3","Q2","Q1"]:
    print(f"For quarter {quarter}")
    full_data = get_data_(
        ticker="TSLA",
        year="2022",
        filing_type="10-Q",
        include_amends=False,
        quarters=quarter
    )
    # print(full_data)
    for data in full_data['TSLA']:
        print(data['year'])
    print("-"*100)

For quarter ALL
Started for TSLA
202209
202206
202203
----------------------------------------------------------------------------------------------------
For quarter Q3
Started for TSLA
202209
----------------------------------------------------------------------------------------------------
For quarter Q2
Started for TSLA
202209
----------------------------------------------------------------------------------------------------
For quarter Q1
Started for TSLA
202203
----------------------------------------------------------------------------------------------------


In [1]:
from data_loader import load_documents

post_process_docs, post_process_metadata = load_documents(
    "AAPL",
    "2021",
    "10-Q",
    include_amends=False,
    num_workers=8,
    quarters="Q2"
)

Started for AAPL


In [2]:
post_process_metadata

[{'full_metadata': 'AAPL_202106_FINANCIAL_STATEMENTS_10-Q'},
 {'full_metadata': 'AAPL_202106_MANAGEMENT_DISCUSSION_10-Q'},
 {'full_metadata': 'AAPL_202106_MARKET_RISK_DISCLOSURES_10-Q'},
 {'full_metadata': 'AAPL_202106_CONTROLS_AND_PROCEDURES_10-Q'},
 {'full_metadata': 'AAPL_202106_LEGAL_PROCEEDINGS_10-Q'},
 {'full_metadata': 'AAPL_202106_RISK_FACTORS_10-Q'},
 {'full_metadata': 'AAPL_202106_USE_OF_PROCEEDS_10-Q'},
 {'full_metadata': 'AAPL_202106_DEFAULTS_10-Q'},
 {'full_metadata': 'AAPL_202106_MINE_SAFETY_10-Q'},
 {'full_metadata': 'AAPL_202106_OTHER_INFORMATION_10-Q'}]

In [2]:
# from summarize import generate_summary

# generate_summary(
#     "AAPL","2021","10-Q",True,8,"Q2"
# )